In [1]:
# DO THE FOLLOWING:
# cd anaconda3 (or w./e your anaconda folder is called)
# source bin/activate
# conda create -n yourenvname python=3.5 anaconda
# source activate yourenvname
# conda install -n yourenvname -c conda-forge tensorflow=1.0
# pip install pillow==4.0.0
# pip install opencv-python==3.4.2.17
# pip install tqdm 

# coding: utf-8

# # CS - 587 : Exercise 3a
# ## Scope:
# The goal of this assignment is to get familiar with fine-tunning in a new dataset a Convolutional Neural Network (CNN) that has been trained in another dataset, taking advantage of transfer learning.
# 
# In your assignment you will be fine-tunning AlexNet a popular CNN architecture, that has been pretrained on ImageNet dataset. Your network will be finetuned for the task of recognizing art painting categories in a large dataset of art painting images, known as Wikiart.
# 
# The WikiArt dataset, which consists of 4000 images of paintings of arbitrary sizes from 10 different styles - Baroque, Realism, Expressionism, etc.
# 
# 

# In[4]:


import os
os.environ['TF_CPP_MIN_VLOG_LEVEL']='3'
import tensorflow as tf
import numpy as np
from models.AlexNet import AlexNet
from Utilities.datagenerator import ImageDataGenerator
import sys
from datetime import datetime
from tqdm import tqdm
#import urllib
import urllib.request


"""
Configuration settings
"""
weight_path= os.path.join('weights','bvlc_alexnet.npy')
general_path_weights = os.path.join('weights')

# Create parent path if it doesn't exist
if not os.path.isdir(general_path_weights): 
    os.mkdir(general_path_weights)
    
def reporthook(blocknum, blocksize, totalsize):
    readsofar = blocknum * blocksize
    if totalsize > 0:
        percent = readsofar * 1e2 / totalsize
        s = "\r%5.1f%% %*d / %d" % (
            percent, len(str(totalsize)), readsofar, totalsize)
        sys.stderr.write(s)
        if readsofar >= totalsize: # near the end
            sys.stderr.write("\n")
    else: # total size is unknown
        sys.stderr.write("read %d\n" % (readsofar,))   
    
if os.path.isfile(weight_path) == False:
    print('Went to download weights for AlexNet, 230Mb from your disk will be put to good use! ')
    print('Relax...')
    weight_file = urllib.request.urlretrieve('http://www.cs.toronto.edu/~guerzhoy/tf_alexnet/bvlc_alexnet.npy',
                                             os.path.join('weights/bvlc_alexnet.npy'),
                                             reporthook)
    print('Done with weights!')
else:
    print('GOOD TO GO! Weights already downloaded and stored!')
    

tf.logging.set_verbosity(tf.logging.ERROR)

# Path to the textfiles for the trainings and validation set
training_dirname = os.path.join('Utilities','data', 'train.txt')

val_dirname = os.path.join('Utilities','data', 'test.txt')

# Path for tf.summary.FileWriter and to store model checkpoints
general_path = os.path.join('finetune_alexnet')
filewriter_path = os.path.join('finetune_alexnet','wikiart')
checkpoint_path = os.path.join('finetune_alexnet','CheckPoint')

# Create parent path if it doesn't exist
if not os.path.isdir(general_path): 
    os.mkdir(general_path)
# Create parent path if it doesn't exist
if not os.path.isdir(filewriter_path): os.mkdir(filewriter_path)
# Create parent path if it doesn't exist
if not os.path.isdir(checkpoint_path): os.mkdir(checkpoint_path)


# In[ ]:


# Learning params
learning_rate = 0.01
num_epochs = 5
batch_size = 32

# Network params
dropout_rate = 0.5
num_classes = 10

#---------------LOOK AT ME----------------------------------------------------

train_layers = ['fc8', 'fc7'] # skip_layer : Change me if you want to try stuff

#------------------------------------------------------------------------------

# # Pretrained Model
# For all of our image generation experiments, we will start with a convolutional neural network which was pretrained to perform image classification on ImageNet. We can use any model here, but for the purposes of this assignment we will use AlexNet

# In[ ]:


# TF placeholder for graph input and output
x = tf.placeholder(tf.float32, [batch_size, 227, 227, 3])
y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)

# Initialize model
model = AlexNet(x, keep_prob, num_classes, train_layers)

# List of trainable variables of the layers we want to train
var_list = [v for v in tf.trainable_variables() if v.name.split('/')[0] in train_layers]

# Link variable to model output
score = model.fc8

GOOD TO GO! Weights already downloaded and stored!


In [2]:
model

In [3]:
################################################################################################
# TODO: Implement the (a) losss function (Soft-max Cross Entropy), (b) the optimization        #
# process using Gradient Descent, (c) accuracy (using argmax). Create summaries in tensorboard #
# for the loss, the gradients of trainable variables (histogram form) and the accuracy.        #
#                                                                                              # 
# Hint: in order to take the gradients per variable use tf.gradient(,)                         #
# https://www.tensorflow.org/api_docs/python/tf/gradients  -> based on the loss                #
################################################################################################    

# define loss 
with tf.name_scope("cross_ent"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = score, labels = y))

with tf.name_scope("train"):
    # define gradients d(loss)/d(w), where w the weights in var_list
    gradients = tf.gradients(loss, var_list) # graidents[0]: (4096,4096) weights of 'fc7'
                                            # gradients[1]: (4096,) bias of 'fc7'
                                            # gradients[2]: (4096,10) weights of 'fc8'
                                            # gradients[3]: (10,) bias of 'fc8'
    # reform gradients as list of tuples: 
    # [(gradients[0], 'fc7'), (gradients[1],'fc7'), (gradients[2],'fc8'), (gradients[3],'fc8')]
    gradients = list(zip(gradients, var_list))

    # define training optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.apply_gradients(grads_and_vars=gradients)
    #train_op = optimizer.minimize(loss)

# add the gradients-tuples of the variables we train to summary
for grad, var in gradients:
    tf.summary.histogram(var.name + '/gradient', grad)
    
# add the variables we train to the summary
for var in var_list:
    tf.summary.histogram(var.name, var)
    
# add loss to the summary
tf.summary.scalar('cross_entropy', loss)

with tf.name_scope("accuracy"):
    # define accuracy
    pred_correct = tf.equal(tf.argmax(score,1), tf.argmax(y,1))
    pc_float32 = tf.cast(pred_correct, tf.float32)
    accuracy = tf.reduce_mean(pc_float32)

# add accuracy to summary
tf.summary.scalar('accuracy', accuracy)

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [4]:
gradients

[(<tf.Tensor 'train/gradients/fc7/fc7/MatMul_grad/MatMul_1:0' shape=(4096, 4096) dtype=float32>,
  <tensorflow.python.ops.variables.Variable at 0x7f5c6e94c2e8>),
 (<tf.Tensor 'train/gradients/fc7/fc7_grad/BiasAddGrad:0' shape=(4096,) dtype=float32>,
  <tensorflow.python.ops.variables.Variable at 0x7f5c6e94c2b0>),
 (<tf.Tensor 'train/gradients/fc8/fc8/MatMul_grad/MatMul_1:0' shape=(4096, 10) dtype=float32>,
  <tensorflow.python.ops.variables.Variable at 0x7f5c3acf79b0>),
 (<tf.Tensor 'train/gradients/fc8/fc8_grad/BiasAddGrad:0' shape=(10,) dtype=float32>,
  <tensorflow.python.ops.variables.Variable at 0x7f5c3acf7978>)]

In [5]:
# Merge all summaries together
merged_summary = tf.summary.merge_all()

# Initialize the FileWriter
writer = tf.summary.FileWriter(filewriter_path)

# Initialize an saver for store model checkpoints
saver = tf.train.Saver()

# Initalize the data generator seperately for the training and validation set
train_generator = ImageDataGenerator(training_dirname, 
                                     horizontal_flip = True, 
                                     shuffle = True)
val_generator = ImageDataGenerator(val_dirname, 
                                   shuffle = False) 

# Get the number of training/validation steps per epoch
train_batches_per_epoch = np.floor(train_generator.data_size / batch_size).astype(np.int16)
val_batches_per_epoch = np.floor(val_generator.data_size / batch_size).astype(np.int16)

In [6]:
# Start Tensorflow session
with tf.Session() as sess:
 
    # Initialize all variables
    sess.run(tf.global_variables_initializer())

    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)

    # Load the pretrained weights into the non-trainable layer
    model.load_initial_weights(sess)

    print("{} Start training...".format(datetime.now()))
    print("{} Open Tensorboard at --logdir {}".format(datetime.now(), 
                                                filewriter_path))
    
    display_step = 3
    # Loop over number of epochs
    for epoch in range(num_epochs):

        print("{} Epoch number: {}".format(datetime.now(), epoch+1))

        step = 1

        for step in tqdm(range(train_batches_per_epoch)):

            # Get a batch of images and labels
            batch_xs, batch_ys = train_generator.next_batch(batch_size)

            ######################################################################################
            #TODO: Run the training operation, print the current loss value write the symmaries.  #
            #      The summarries must be written every 3 batches                                #
            ######################################################################################

            sess.run(train_op, feed_dict = {x: batch_xs, 
                                            y: batch_ys,
                                            keep_prob: dropout_rate})
            # every 3 batches generate summary with the current batch of data
            if step%display_step == 0:
                s = sess.run(merged_summary, feed_dict={x: batch_xs,
                                                       y: batch_ys,
                                                       keep_prob: 1.}) # keep all units, i.e. no dropout
                writer.add_summary(s, epoch*train_batches_per_epoch + step)
                
            step += 1

            #End of this task

        ############################################################
        #TODO: Validate the model on the ENTIRE validation set     #
        #      Print the final validation accuracy of your model   #
        ############################################################   
        
        test_acc = 0.
        test_count = 0
        
        # for all validation batches
        for _ in range(val_batches_per_epoch):
            batch_tx, batch_ty = val_generator.next_batch(batch_size)
            acc = sess.run(accuracy, feed_dict = {x: batch_tx,
                                                  y: batch_ty,
                                                  keep_prob: 1.}) # keep all units
            test_acc += acc
            test_count += 1
        
        test_acc /= test_count # mean accuracy for the validation set
        #print("Validation Accuracy: {:, .4f}".format(datetime.now(), test_acc))
        print("Validation Accuracy: {:.4f}".format(test_acc))

        ##############################################################################
        #                             END OF YOUR CODE                               #
        ##############################################################################

        # Reset the file pointer of the image data generator
        val_generator.reset_pointer()
        train_generator.reset_pointer()

        print("{} Saving checkpoint of model...".format(datetime.now()))  
        

        #save checkpoint of the model
        checkpoint_name = os.path.join(checkpoint_path, 'model_epoch'+str(epoch+1)+'.ckpt')
        save_path = saver.save(sess, checkpoint_name)  

        print("{} Model checkpoint saved at {}".format(datetime.now(), checkpoint_name))



  0%|          | 0/100 [00:00<?, ?it/s]

2020-05-27 01:02:54.578121 Start training...
2020-05-27 01:02:54.578211 Open Tensorboard at --logdir finetune_alexnet/wikiart
2020-05-27 01:02:54.578240 Epoch number: 1


100%|██████████| 100/100 [02:41<00:00,  1.61s/it]


Validation Accuracy: 0.3333
2020-05-27 01:06:02.157828 Saving checkpoint of model...


  0%|          | 0/100 [00:00<?, ?it/s]

...

Validation Accuracy: 0.3307
2020-05-27 01:15:20.754179 Saving checkpoint of model...


  0%|          | 0/100 [00:00<?, ?it/s]

2020-05-27 01:15:24.386467 Model checkpoint saved at finetune_alexnet/CheckPoint/model_epoch4.ckpt
2020-05-27 01:15:24.386757 Epoch number: 5


100%|██████████| 100/100 [02:40<00:00,  1.60s/it]


Validation Accuracy: 0.3802
2020-05-27 01:18:31.993029 Saving checkpoint of model...
2020-05-27 01:18:35.557945 Model checkpoint saved at finetune_alexnet/CheckPoint/model_epoch5.ckpt


In [7]:
# OPEN A TERMINAL
# cd anaconda3
# source bin/acitvate
# source activate youenvname
# tensorboard --logdir=./finetune_alexnet/wikiart